# Lecture 12 – Longitudinal plot

Prepare the notebook with the preambles for the inclusion of pandas, numpy and matplotlib.pyplot.
Import the data file (csv format) into Google Colab through the drive.mount command, import the pylorentz package.

In [ ]:
%pip install -q gdown matplotlib numpy pandas pylorentz

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pylorentz import Momentum4

In [ ]:
import gdown

output_path = gdown.cached_download(
    url="https://drive.google.com/uc?id=1VkG1gYY4qnfqNz3MEUkOKDwcAW4EF4uR",
    path="data/gammapi_2pions_exclusivityCuts.dat",
    md5="6aa2ab1388aa7fc1ce0568e3e972e491",
    quiet=True,
)
data = pd.read_csv(output_path)
data.columns = data.columns.str.strip()
data

Let's prepare the relevant arrays containing the 4-momenta kinematics of the three particle (plus the two in the intial state), both in the laboratory and in the center of mass reference systems:

In [ ]:
# final state
pip = Momentum4(data.E1, data.p1x, data.p1y, data.p1z)
pim = Momentum4(data.E2, data.p2x, data.p2y, data.p2z)
proton = Momentum4(data.E3, data.p3x, data.p3y, data.p3z)

# initial state
n_events = len(data)
zeros = np.zeros(n_events)
ones = np.ones(n_events)
m_proton = 0.93827
gamma = Momentum4(data.pgamma, zeros, zeros, data.pgamma)
target = Momentum4(m_proton * ones, zeros, zeros, zeros)

In [ ]:
# 4-momenta in the scenter of mass system
cm = gamma + target
pip_cm = pip.boost_particle(-cm)
pim_cm = pim.boost_particle(-cm)
proton_cm = proton.boost_particle(-cm)
gamma_cm = gamma.boost_particle(-cm)
target_cm = target.boost_particle(-cm)

A **longitudinal plot** (introduced for the first time [by Van Hove in 1969](https://doi.org/10.1016/0550-3213(69)90133-3)) offers a way to separate the contribution of different reaction production mechanisms. Inspecting a longitudinal plot may suggest particular selections in the longitudinal phase space which can help selecting specific reaction channels, enhancing for instance the contribution of meson versus baryon resonances.
They are based on the assumption that at sufficiently high center-of-mass energies the phase space is reflected mainly by the longitudinal components of the particle momenta, while the transverse components can be neglected. Longitudinal plots are a convenient way to visualize the reaction kinematics of a reaction with three particles in the final state, in the form of 2D scattering plots (for $n$ particles in the final state, the reaction kinematics are visualized in a $(n-1)$ dimensional plane).
For a reaction with a three particles final state one defines a set of polar coordinates $q$ and $\omega$ such that, being $q_1,\; q_2,\; q_3$ the longitudinal momentum components of each of them:

$$
\begin{eqnarray}
q_1 &=& \sqrt{\tfrac{2}{3}} q\sin\omega \\
q_2 &=& \sqrt{\tfrac{2}{3}} q\sin (\omega + \tfrac{2}{3}\pi) \\
q_3 &=& \sqrt{\tfrac{2}{3}} q\sin (\omega + \tfrac{4}{3}\pi) \\
q &=& \sqrt{q_1^2 + q_2^2 + q_3^2}
\end{eqnarray}
$$

In a longitudinal plot the $(X,Y)$ coordinates are defined as follows:

$$
\begin{eqnarray}
X &=& q\cos\omega \\
Y &=& q\sin\omega
\end{eqnarray}
$$

Let's build the cartesian coordinates arrays and make the scatter plot.

In [ ]:
q1 = pip_cm.p_z
q2 = pim_cm.p_z
q3 = proton_cm.p_z
q = np.sqrt(q1**2 + q2**2 + q3**2)

In [ ]:
a = np.sqrt(3 / 2) * q1 / q
filter_ = np.abs(a) <= 1
sin_omega = a[filter_]
cos_omega = (2 * q2 + q1) / np.sqrt(2) / q
cos_omega = cos_omega[filter_]
omega = np.arcsin(sin_omega)
omega = np.select(
    [(cos_omega < 0) & (sin_omega >= 0), True],
    [np.pi - omega, omega - np.pi],
)
X = q[filter_] * cos_omega
Y = q[filter_] * sin_omega

In [ ]:
plt.hist2d(X, Y, bins=100, cmap="rainbow")
plt.plot([-10, 10], [0, 0], color="black")
sixty = np.pi / 3
plt.plot(
    [-10 * np.cos(sixty), 10 * np.cos(sixty)],
    [10 * np.sin(sixty), -10 * np.sin(sixty)],
    color="black",
)
plt.plot(
    x3=[10 * np.cos(sixty), -10 * np.cos(sixty)],
    y3=[10 * np.sin(sixty), -10 * np.sin(sixty)],
    color="black",
)
plt.show()

Try with another input file (see [here](https://drive.google.com/drive/folders/1-Knh70_vLuctCkcg9oIIIIBaX2EPYkea) on Google Drive, for instance the generated MonteCarlo events at 5 or 10 GeV incident momentum) and check the differences. The whole folder can be downloaded as follows:


```python
gdown.download_folder(id="1-Knh70_vLuctCkcg9oIIIIBaX2EPYkea", output="data", quiet=True)
```

Then set `data` to for instance

```python
data = pd.read_csv("data/FurtherFun/gammaP_5GeV_protKPlusKMinusEtamiss.csv")
```

and rerun the notebook.